### [Problem 1] Cross Validation

In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# loading the csv of the dataset
df = pd.read_csv('/home/jules/projects/dic/mldocs/application_train.csv')
df = df.select_dtypes('number')

cleaned_df = df.fillna(0)

cleaned_df = cleaned_df[cleaned_df.columns[~cleaned_df.isnull().all()]]

y = cleaned_df['TARGET']
X = cleaned_df.drop(['TARGET'], axis=1)

X = X.to_numpy()

kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

TRAIN: [153756 153757 153758 ... 307508 307509 307510] TEST: [     0      1      2 ... 153753 153754 153755]
TRAIN: [     0      1      2 ... 153753 153754 153755] TEST: [153756 153757 153758 ... 307508 307509 307510]


In [2]:
# Standardizing the data

scaler = StandardScaler()
scaler.fit(X_train)
X_train_trans = scaler.transform(X_train)
X_test_trans = scaler.transform(X_test)

### [Problem 2] Grid search

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# checking which model and params are best
model_params = {
    'random_forest':{
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [1,5,10]
        }
    },
    'logic_regression':{
        'model': LogisticRegression(solver="liblinear",multi_class="auto"),
        'params': {
            'C': [1,5,10]
        }
    }
}

# prepare an array to store the scores
scores = []

for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'], return_train_score=False)
    clf.fit(X_train_trans,y_train)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

best_model_params = pd.DataFrame(scores,columns=['model','best_score','best_params'])
best_model_params

,model,best_score,best_params
0,random_forest,0.915457,{'n_estimators': 10}
1,logic_regression,0.918364,{'C': 1}


### [Problem 3] Survey from Kaggle Notebooks

* find a good combination of hyperparameters by using Random search 
* Hyperparameter Tuning using Grid search
* Gradient Boosting Machine
* Using one type of data
* Early stopping

### [Problem 4] Creating a model with high generalization performance

In [8]:
import lightgbm as lgb

# an instance of the model
model = lgb.LGBMClassifier()

# default params
default_params = model.get_params()

# number of folds
N_FOLDS = 5

# dataset
train_set = lgb.Dataset(data = X_train)

cv_results = lgb.cv(default_params, train_set, num_boost_round = 10000, early_stopping_rounds = 100, metrics = 'auc', nfold = N_FOLDS, seed = 42)

print('The maximum validation ROC AUC was: {:.5f}.'.format(cv_results['auc-mean'][-1]))
print('The optimal number of boosting rounds (estimators) was {}.'.format(len(cv_results['auc-mean'])))

meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

1. imported the whole dataset
2. create a subset of only numbers
3. split the data using kfold
4. use gridsearchCV to find the best model and params to fine tune my classiffiers
5. After finding the results i tested the LGBM classifier and checked it ROC which is good


### [Problem 5] Final model selection

In [13]:
test_df = pd.read_csv('/home/jules/projects/dic/mldocs/application_test.csv')

#remove  empy data(null)
test_cleaned_df = test_df.fillna(0)

# separating them into variables
test_X = test_cleaned_df.select_dtypes('number')

#   data standardizing
test_scaler = StandardScaler()
test_X_test_trans = scaler.fit_transform(test_X)

# predicting
test_reg_pred = clf.predict(test_X_test_trans)

kgl_submission = pd.concat([test_df['SK_ID_CURR'], pd.Series(test_reg_pred, name='TARGET')], axis=1)
kgl_submission.to_csv('kgl_sub.csv', index=False)

<img src="kgl_sub.png"/>